In [1]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, f_regression

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
%run ../common_code.ipynb

In [5]:
#https://www.kaggle.com/datasets/camnugent/california-housing-prices
data = pd.read_csv("../../datasets/abalone.data", sep=',', names=['sex', 'length', 'diameter', 'height', 'W_weight', 'S_weight', 'viscera_weight', 'shell_weight', 'rings'])
data.drop(columns=["sex"], inplace=True)
n_features = data.shape[1]-1
n = data.shape[0]
target = "rings"
features = data.columns.difference([target]).tolist()

data.head(10)

,length,diameter,height,W_weight,S_weight,viscera_weight,shell_weight,rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
5,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8
6,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,20
7,0.545,0.425,0.125,0.7680,0.2940,0.1495,0.260,16
8,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.165,9
9,0.550,0.440,0.150,0.8945,0.3145,0.1510,0.320,19


In [7]:
data.describe()

,length,diameter,height,W_weight,S_weight,viscera_weight,shell_weight,rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [8]:
data_corr = add_extra_features(data, features, extra_size=0.5, mode='random')

In [9]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42)

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42)

features_corr = list(data_corr.columns.difference([target]))

In [10]:
features_corr

['S_weight',
 'W_weight',
 'diameter',
 'extra_001',
 'extra_002',
 'extra_003',
 'extra_004',
 'extra_005',
 'extra_006',
 'extra_007',
 'height',
 'length',
 'shell_weight',
 'viscera_weight']

In [11]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    regr = GradientBoostingRegressor(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    regr.fit(data_dict['train'][features], data_dict['train'][target])
    return score_regr(regr, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [12]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
best_base_params

{'n_estimators': 34, 'max_depth': 8, 'learning_rate': 0.08282245617333785}

In [14]:
model_corr = GradientBoostingRegressor(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='ape_error', task='regr')

In [15]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["ape_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,length_0,length_1,length_2,diameter_0,diameter_1,diameter_2,height_0,height_1,height_2,W_weight_0,...,extra_005_2,extra_006_0,extra_006_1,extra_006_2,extra_007_0,extra_007_1,extra_007_2,ape_error_0,ape_error_1,ape_error_2
0,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,0,1,1,0,0


In [16]:
disc_data_corr['ape_error'].value_counts()

ape_error
0    304
1    175
2     54
Name: count, dtype: int64

In [17]:
feat_to_grad_corr = {}
for feat in features_corr+['ape_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'mape': lambda x, y: m.mean_absolute_percentage_error(x,y)*100,
    'r2_score': lambda x, y: m.r2_score(x, y)*100
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'ape_error', categories_corr, score=BicScore)

2025-05-05 19:58:16,601 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,603 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,604 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8


2025-05-05 19:58:16,605 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,607 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,608 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,611 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,613 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,615 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,615 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,616 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,617 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-05 19:58:16,618 | ERROR    | GraphUtil

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [18]:
gbn_fs.get_info()

,name,node_type,data_type,parents,parents_types
0,diameter_0,Discrete,disc,[],[]
1,extra_001_0,Discrete,disc,[],[]
2,extra_001_1,Discrete,disc,[],[]
3,extra_001_2,Discrete,disc,[],[]
4,extra_002_0,Discrete,disc,[],[]
5,extra_002_1,Discrete,disc,[],[]
6,extra_002_2,Discrete,disc,[],[]
7,extra_003_1,Discrete,disc,[],[]
8,extra_004_1,Discrete,disc,[],[]
9,extra_005_0,Discrete,disc,[],[]


In [19]:
_df=gbn_fs.get_info()
_df[_df['name'].apply(lambda x: 'ape_error' in x.name)]

,name,node_type,data_type,parents,parents_types
39,ape_error_0,Discrete,disc,[viscera_weight_2],[disc]
40,ape_error_1,Discrete,disc,[viscera_weight_2],[disc]
44,ape_error_2,Discrete,disc,"[shell_weight_1, extra_004_1]","[disc, disc]"


# Univariate Feature selector

In [20]:
univ_sel = SelectKBest(f_regression, k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [21]:
rfe_sel = RFE(estimator=DecisionTreeRegressor(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [24]:
lsvc = Lasso(alpha=0.015, random_state=42)
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [25]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['S_weight',
 'W_weight',
 'diameter',
 'extra_001',
 'extra_002',
 'extra_003',
 'extra_006',
 'extra_007',
 'shell_weight']

In [26]:
len(lasso_support)

9

## GBN selector

In [27]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'ape_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 14/14 [00:00<00:00, 20.55it/s]


In [28]:
feat_to_grad_corr

{'S_weight': ['S_weight_0', 'S_weight_1', 'S_weight_2'],
 'W_weight': ['W_weight_0', 'W_weight_1', 'W_weight_2'],
 'diameter': ['diameter_0', 'diameter_1', 'diameter_2'],
 'extra_001': ['extra_001_0', 'extra_001_1', 'extra_001_2'],
 'extra_002': ['extra_002_0', 'extra_002_1', 'extra_002_2'],
 'extra_003': ['extra_003_0', 'extra_003_1', 'extra_003_2'],
 'extra_004': ['extra_004_0', 'extra_004_1', 'extra_004_2'],
 'extra_005': ['extra_005_0', 'extra_005_1', 'extra_005_2'],
 'extra_006': ['extra_006_0', 'extra_006_1', 'extra_006_2'],
 'extra_007': ['extra_007_0', 'extra_007_1', 'extra_007_2'],
 'height': ['height_0', 'height_1', 'height_2'],
 'length': ['length_0', 'length_1', 'length_2'],
 'shell_weight': ['shell_weight_0', 'shell_weight_1', 'shell_weight_2'],
 'viscera_weight': ['viscera_weight_0',
  'viscera_weight_1',
  'viscera_weight_2'],
 'ape_error': ['ape_error_0', 'ape_error_1', 'ape_error_2']}

## GBN selector run

In [29]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc', task='regr')

100%|██████████| 14/14 [00:24<00:00,  1.72s/it]


In [30]:
gbn_res.head()

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,14,-,16.031,52.25,0,0
1,13,extra_001,15.691,53.367,-2.122,2.137
2,12,extra_002,15.527,53.215,-3.145,1.847
3,11,extra_003,15.696,52.285,-2.091,0.067
4,10,extra_005,15.681,52.323,-2.181,0.14


In [31]:
get_best_record(gbn_res, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
7,7,extra_004,15.305,54.395,-4.528,4.105


In [32]:
get_best_record(gbn_res, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
6,8,extra_007,15.345,54.41,-4.28,4.134


In [33]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc', task='regr')
df_corr_excluded_asc_univ.head()

100%|██████████| 14/14 [00:36<00:00,  2.63s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,14,-,15.961,52.014,0,0
1,13,shell_weight,16.432,49.123,2.949,-5.558
2,12,height,16.299,51.846,2.12,-0.323
3,11,diameter,16.413,51.462,2.833,-1.062
4,10,length,16.804,49.79,5.284,-4.277


In [34]:
get_best_record(df_corr_excluded_asc_univ, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,14,-,15.961,52.014,0,0


In [35]:
get_best_record(df_corr_excluded_asc_univ, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,14,-,15.961,52.014,0,0


In [36]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc', task='regr')
df_corr_excluded_rfe.head()

100%|██████████| 14/14 [00:29<00:00,  2.09s/it]


,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
0,14,-,16.003,52.652,0,0
1,13,extra_007,16.122,52.33,0.742,-0.611
2,12,extra_001,15.635,53.677,-2.299,1.947
3,11,diameter,15.835,52.657,-1.047,0.01
4,10,extra_004,16.012,52.521,0.058,-0.248


In [37]:
get_best_record(df_corr_excluded_rfe, 'mape', mode='min')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
6,8,extra_006,15.544,54.309,-2.866,3.146


In [38]:
get_best_record(df_corr_excluded_rfe, 'r2_score')

,n_features,last_excluded,mape,r2_score,"mape_change, %","r2_score_change, %"
6,8,extra_006,15.544,54.309,-2.866,3.146


In [39]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict, task='regr')

[15.88, 53.887]

In [ ]:
#logging mape (probably woth to be negative mape)
0.5 random, base 18.467, gbn 17.877, univ 18.467, rfe 18.103 , lasso: 17.89
0.75 random, base 19.137, gbn 18.346, univ 19.137, rfe 18.239 , lasso: 18.338
0.5 noisy, base 18.337, gbn 17.927, univ 18.337, rfe  18.109, lasso: 18.221
0.75 noisy, base 18.442, gbn 17.960, univ 18.442, rfe  18.185, lasso: 18.586
0.5 prod, base 17.865, gbn 17.865, univ 17.865, rfe 17.802, lasso: 17.762
0.75 prod, base 18.442, gbn 18.417, univ 18.382, rfe 18.029, lasso: 18.585


In [ ]:
# logging r2_score
0.5 random, base 81.675, gbn 82.667, univ 81.675, rfe  82.27, lasso: 82.616 
0.75 random, base 80.482, gbn 81.832, univ 80.619, rfe  82.188, lasso: 81.806
0.5 noisy, base 81.813, gbn 82.479, univ 81.863, rfe  82.047, lasso: 81.88
0.75 noisy, base 81.637, gbn 82.039, univ 81.596, rfe  81.97, lasso: 81.392
0.5 prod, base 81.97, gbn 81.97, univ 82.176, rfe 82.438, lasso: 82.363
0.75 prod, base 81.637, gbn 81.637, univ 81.704, rfe 82.364, lasso: 80.941


In [ ]:
# #logging mape
# 0.5 random, base 17.877, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
# 0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

# 0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
# 0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

# 0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
# 0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
